# 7/23 情報工学工房
今日の内容<br>
解説編

- PyTorchのDatasetについて

実習編

- 100本ノック第8章 70のプログラム(70.pyの「#todo」と書かれた行を埋めてプログラムを完成させる)
- 100本ノック第8章 71～74，77をまとめたプログラムの完成(77.pyを完成させる)

Bisiness
Technology
Medical

xは
yは正解データ。クラスのラベル
nは学習データの事例の数
文のベクトルを求めるには
埋め込みベクトルを足し合わせて平均を取る
データセットについて
自分でデータセットを作る
data_loaderを使うとバッチ処理が楽、
自作のデータセットの作り方

isとかareは考慮しないだったり
今は順番考慮していない

1. Datasetというクラスを継承する（PyTorchに入っている）
2. init,len,getitemを作る(init データを返す)(len データサイズを返す)(getitem 引数としてidxインデックスが渡される。idx番目のデータを返すようにする)
<br>
インデックスが偶数 / 奇数で分けたり対話処理の不正解は特殊

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/kobo2020
BetterRnnlm.pkl  data70/                             PyTorchTutorial1.ipynb
better_rnnlm.py  dataset/                            repo6.ipynb
cbow_params.pkl  GoogleNews-vectors-negative300.bin  repo7.ipynb
cbow.py          negative_sampling_layer.py          repo8.ipynb
ch04/            neko.txt                            simple_rnnlm.py
common/          __pycache__/


In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.time_layers import *
from common.np import *  # import numpy as np
from common.base_model import BaseModel


class BetterRnnlm(BaseModel):

    def __init__(self, vocab_size=10000, wordvec_size=650,
                 hidden_size=650, dropout_ratio=0.5):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx1 = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh1 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b1 = np.zeros(4 * H).astype('f')
        lstm_Wx2 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_Wh2 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b2 = np.zeros(4 * H).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.layers = [
            TimeEmbedding(embed_W),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, stateful=True),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx2, lstm_Wh2, lstm_b2, stateful=True),
            TimeDropout(dropout_ratio),
            TimeAffine(embed_W.T, affine_b)  # weight tying!!
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layers = [self.layers[2], self.layers[4]]
        self.drop_layers = [self.layers[1], self.layers[3], self.layers[5]]

        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs, train_flg=False):
        for layer in self.drop_layers:
            layer.train_flg = train_flg

        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts, train_flg=True):
        score = self.predict(xs, train_flg)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()


In [ ]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   3882      0 --:--:-- --:--:-- --:--:--  3882
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common import config
config.GPU = True

from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb


# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size,
                time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('valid perplexity: ', ppl)

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr

    model.reset_state()
    print('-' * 50)


# テストデータでの評価
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)


| epoch 1 |  iter 1 / 1327 | time 3[s] | perplexity 10000.00
| epoch 1 |  iter 21 / 1327 | time 67[s] | perplexity 3706.32
| epoch 1 |  iter 41 / 1327 | time 132[s] | perplexity 2014.43
| epoch 1 |  iter 61 / 1327 | time 197[s] | perplexity 1299.31
| epoch 1 |  iter 81 / 1327 | time 261[s] | perplexity 1076.62
| epoch 1 |  iter 101 / 1327 | time 325[s] | perplexity 852.96
| epoch 1 |  iter 121 / 1327 | time 389[s] | perplexity 794.85
| epoch 1 |  iter 141 / 1327 | time 452[s] | perplexity 737.02
| epoch 1 |  iter 161 / 1327 | time 515[s] | perplexity 685.62
| epoch 1 |  iter 181 / 1327 | time 579[s] | perplexity 683.15
| epoch 1 |  iter 201 / 1327 | time 642[s] | perplexity 589.71
| epoch 1 |  iter 221 / 1327 | time 705[s] | perplexity 579.71
| epoch 1 |  iter 241 / 1327 | time 768[s] | perplexity 518.68
| epoch 1 |  iter 261 / 1327 | time 832[s] | perplexity 544.64
| epoch 1 |  iter 281 / 1327 | time 895[s] | perplexity 514.00
| epoch 1 |  iter 301 / 1327 | time 958[s] | perplexity 44

# 感想
今回は、前回のRNNを繰り返すニューラルネットワークの問題点を改善したLSTMを繰り返すニューラルネットワークについて学習した。LSTMは、過去の情報の重要度に応じて記憶する濃度？のようなものを設定して勾配爆発または勾配消失が起きないようにした。感想としては人間の脳の動きに近づいているような感じがした。（もちろんまだ脳の中身は完全に解明されてないからこんな感じなのかなくらいのイメージではあるけれど）最後の3つの工夫では、一つ目の多層化と二つ目のDropoutは理解はしたが、三つ目の重みの共有があまりよくわからなかった。<br>
今回の章のものからGoogle翻訳の話とか出てきて現在使われているシステムに近づいてきていると感じた。
<br>学習は時間がかかるのでもっと早くから実行時間を確認してやらないといけないと思った。今回GPUで実行したのだが5時間で終わらなかった。<br>
Google Colaboratryで実行したが最後までできなかった。
7世代目までの結果しかないが、100語くらいまで絞れた。この後最後までやると75語まで絞れるらしい。<br>

言語処理100本ノックの方は、テンソルとリストの変換でつまった。テンソルとnumpyの変換はあるので一度リストをnumpyに変換してからテンソルに変換してなんとかはなった。